In [ ]:
!pip uninstall transformers -y
!pip install transformers[torch] datasets

Found existing installation: transformers 4.41.1
Uninstalling transformers-4.41.1:
  Successfully uninstalled transformers-4.41.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cubla

In [1]:
# prompt: google mount

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
from transformers import AutoModel, AutoTokenizer, DataCollatorWithPadding, TrainingArguments, Trainer
import torch.nn as nn
import torch
from transformers.modeling_outputs import SequenceClassifierOutput

# Define model
class BertCNNClassifier(nn.Module):
    def __init__(self, bert_model, num_labels):
        super(BertCNNClassifier, self).__init__()
        self.bert = bert_model
        self.cnn1 = nn.Conv1d(1536, 512, kernel_size=2, padding=1)
        self.cnn2 = nn.Conv1d(512, 64, kernel_size=2, padding=1)
        self.relu = nn.ReLU()
        self.global_avg_pool = nn.AdaptiveAvgPool1d(1)
        self.classifier = nn.Linear(64, num_labels)
        self.num_labels = num_labels

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, labels=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        all_hidden_states = torch.stack(outputs.hidden_states)
        concatenate_pooling = torch.cat((all_hidden_states[-1], all_hidden_states[-4]), dim=-1)
        concatenate_pooling = concatenate_pooling.permute(0, 2, 1)

        x = self.relu(self.cnn1(concatenate_pooling))
        x = self.relu(self.cnn2(x))
        x = self.global_avg_pool(x).squeeze(-1)
        logits = self.classifier(x)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )


In [7]:
import torch.nn.functional as F
from safetensors.torch import load_file as safe_load_file

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# BERT 모델 로드 및 커스텀 모델 초기화
klue_bert = AutoModel.from_pretrained('klue/bert-base', output_hidden_states=True)
model = BertCNNClassifier(bert_model=klue_bert, num_labels=60)
model.to(device)

# checkpoint-18800에서 모델 가중치 로드 (safetensors 사용)
checkpoint_path = "checkpoint-18800/model.safetensors"
state_dict = safe_load_file(checkpoint_path)
model.load_state_dict(state_dict)

# 모델을 평가 모드로 설정
model.eval()


BertCNNClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [8]:
id2label = {
    0: "분노",
    1: "툴툴대는",
    2: "좌절한",
    3: "짜증내는",
    4: "방어적인",
    5: "악의적인",
    6: "안달하는",
    7: "구역질 나는",
    8: "노여워하는",
    9: "성가신",
    10: "슬픔",
    11: "실망한",
    12: "비통한",
    13: "후회되는",
    14: "우울한",
    15: "마비된",
    16: "염세적인",
    17: "눈물이 나는",
    18: "낙담한",
    19: "환멸을 느끼는",
    20: "불안",
    21: "두려운",
    22: "스트레스 받는",
    23: "취약한",
    24: "혼란스러운",
    25: "당혹스러운",
    26: "회의적인",
    27: "걱정스러운",
    28: "조심스러운",
    29: "초조한",
    30: "상처",
    31: "질투하는",
    32: "배신당한",
    33: "고립된",
    34: "충격 받은",
    35: "가난한, 불우한",
    36: "희생된",
    37: "억울한",
    38: "괴로워하는",
    39: "버려진",
    40: "당황",
    41: "고립된(당황한)",
    42: "남의 시선을 의식하는",
    43: "외로운",
    44: "열등감",
    45: "죄책감",
    46: "부끄러운",
    47: "혐오스러운",
    48: "한심한",
    49: "혼란스러운(당황한)",
    50: "기쁨",
    51: "감사하는",
    52: "신뢰하는",
    53: "편안한",
    54: "만족스러운",
    55: "흥분",
    56: "느긋",
    57: "안도",
    58: "신이 난",
    59: "자신하는"
}

In [19]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F

# 입력 텍스트 정의
input_texts = ["""
너 없는 지금도 눈부신 하늘과
눈부시게 웃는 사람들
나의 헤어짐을 모르는 세상은
슬프도록 그대로인데
시간마저 데려가지 못하게
나만은 널 보내지 못했나봐
가시처럼 깊게 박힌 기억은
아파도 아픈줄 모르고
그대 기억이 지난 사랑이
내 안을 파고 드는 가시가 되어
제발 가라고 아주 가라고
애써도 나를 괴롭히는데
아픈만큼 너를 잊게 된다면
차라리 앓고 나면 그만인데
가시처럼 깊게 박힌 기억은
아파도 아픈 줄 모르고
제발 가라고 아주 가라고
애써도 나를 괴롭히는데
너무 사랑했던 나를
크게 두려웠던 나를
미치도록 너를 그리워했던
날 이제는 놓아줘
보이지 않아 내 안에 숨어
잊으려 하면 할수록 더 아파와
제발 가라고 아주 가라고
애써도 나를 괴롭히는데
""", """옛날 옛날에 남양주시 와부읍
덕소리에서 자란 한 소년이 있었어요
아이는 주머니에 아무것도 없었지만
어머니의 사랑은 가득해
항상 웃음 짓던 소년이였죠
어느 날 그 소년 앞에
금을 두른 부자가 나타났답니다
부자는 그에게 물었어요
너에게 미래를 줄 테니
지금 이 순간을 나와 바꾸지 않겠니?
나 평생 꿈만을 꿨죠
알잖아요 꿈은 안 들잖아 돈
나 이 순간을 나 평생 떠올렸죠
친구들이 대학을 갈 때
난 한강에 가서 술을 마셨네
되뇌이면서 '세상은 날 싫어해'
그렇지 그렇지 그럴만했어
그때는 몰라 그리고 애써
알려고 하지도 않잖아
눈물을 흘렸지 내 방 속에서
눈물 흘려 여의도에서
두 눈물 맛이 달라 아
빌었어
빌었어 밤마다
이럴 땐 술김에 오그라들게
두 손을 모으고 말야
Oh oh 빌었어
빌었어 밤마다
나 무교잖아
근데 하늘에다가
비는 걸 보면 있나 봐
내 소원을 들어줄 어떤 이 (hey)""","""
니가 없는 거리에는 내가 할 일이 없어서
마냥 걷다 걷다 보면 추억을 가끔 마주치지
떠오르는 너의 모습 내 살아나는 그리움 한 번에
참 잊기 힘든 사람이란 걸 또 한 번 느껴지는 하루
어디쯤에 머무는지 또 어떻게 살아가는지
걷다 보면 누가 말해줄 것 같아
이 거리가 익숙했던 우리 발걸음이 나란했던
그리운 날들 오늘 밤 나를 찾아온다
널 그리는 널 부르는 내 하루는
애태워도 마주친 추억이 반가워
날 부르는 목소리에 돌아보면
텅 빈 거리 어느새 수많은 니 모습만 가득해
막다른 길 다다라서 낯익은 벽 기대 보면
가로등 속 환히 비춰지는 고백하는 니가 보여
떠오르는 그때 모습 내 살아나는 설레임 한 번에
참 잊기 힘든 순간이란 걸 또 한 번 느껴지는 하루
아직 나를 생각할지 또 그녀도 나를 찾을지
걷다 보면 누가 말해줄 것 같아
이 거리가 익숙했던 우리 발걸음이 나란했던
그리운 날들 오늘 밤 나를 찾아온다
널 그리는 널 부르는 내 하루는
애태워도 마주친 추억이 반가워
날 부르는 목소리에 돌아보면
텅 빈 거리 어느새 수많은 니 모습만 가득해
부풀은 내 가슴이 밤하늘에 외쳐본다
이 거리는 널 기다린다고
널 그리는 널 부르는 내 하루는
애태워도 마주친 추억이 반가워
날 부르는 목소리에 돌아보면
텅 빈 거리 어느새 수많은 니 모습만
가득해
""", """
아침에 눈을 떴을 때 너를
길을 걷다 멍하니 너를
지금은 내 곁에 없는 너를
그리워하네 바보처럼
나보다 행복하기를 바래
내 생각하지 않기를 바래
더 좋은 사람 만나길 바래
다시는 내게 올 수 없게
안개처럼 사라져 간 다시 못 올 그 지난날
함께한 추억 모두 흘려보낼게
널 잊어야 해 힘들어도
널 지워야 해 기억 속에서
네가 떠난 후에 난 죽을 것같이 아파도
두 번 다시 울지 않을게
잊을게 잊을게
아직도 휴대폰에 네 이름
지우지도 못하고 있어
전화기 들고 한참을 서서
널 생각하네 바보처럼
안개처럼 사라져 간 다시 못 올 그 지난날
함께한 추억 모두 흘려보낼게
널 잊어야 해 힘들어도
널 지워야 해 기억 속에서
네가 떠난 후에 난 죽을 것같이 아파도
다시는 너를 찾지 않아
아침에 눈을 떴을 때 너를 아직도
길을 걷다 멍하니 너를
지금은 내 곁에 없는 너를
그리워하네 바보처럼

""", """
너와 나 둘이 정신없이 가는 곳
정처 없이 가는 곳, 정해지지 않은 곳
거기서 우리 (whoa-whoa) 서로를 재워주고 (whoa-whoa)
서로를 깨워주고 (whoa-whoa) 서로를 채워주고 (whoa-whoa)
Excuse me 잠시만 아직까진 우린 남
하지만 조만간 중독성을 자랑하는 장난감
지금 이 느낌적인 느낌이 통하는 느낌
녹아버릴 아이스크림
지금이 우리에게는 꿈이야
너와 나 둘이서 추는 춤이야
기분은 미친 듯이 예술이야
Ooh-eh-oh, ooh-eh-oh, ooh-eh-oh
하늘을 날아가는 기분이야
죽어도 상관없는 지금이야
심장은 터질 듯이 예술이야
Ooh-eh-oh, ooh-eh-oh, ooh-eh-oh
예술이야 (예술이야), 예술이야 (예술이야)
예술이야 (예술이야), 이런 날이 올 줄이야
예술이야 (예술이야), 예술이야 (예술이야)
Ooh-eh-oh, ooh-eh-oh, ooh-eh-oh
너와 나 둘이 (whoa-whoa) 밤새 잔을 부딪혀 (whoa-whoa)
밤새 뺨을 부비며 (whoa-whoa) 밤새도록 둘이서 (whoa-whoa)
눈이 점점 풀린다
니도 따라 풀린다
끌린다, 너에 대한 수수께끼가 풀린다
지금 이 춤에 너의 가빠진 숨에
수줍음에 you know what I mean
지금이 우리에게는 꿈이야
너와 나 둘이서 추는 춤이야
기분은 미친 듯이 예술이야
Ooh-eh-oh, ooh-eh-oh, ooh-eh-oh
하늘을 날아가는 기분이야
죽어도 상관없는 지금이야"""]

# 모델과 토크나이저 로드
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')
# 입력 텍스트를 토큰화
inputs = tokenizer(input_texts, padding=True, truncation=True, return_tensors="pt").to(device)
model.to(device)

# 예측 수행
with torch.no_grad():
    outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
    logits = outputs.logits
    probabilities = F.softmax(logits, dim=-1)

# 결과 출력 및 시각화
for i, text in enumerate(input_texts):
    print(f"Input: {text}")
    # print(f"Prediction: {id2label[predictions[i].item()]} (Probability: {probabilities[i][predictions[i]].item():.4f})")

    top_probabilities, top_indices = torch.topk(probabilities[i], 10)
    for j in range(10):
        print(f"Top {j+1} Prediction: {id2label[top_indices[j].item()]} (Probability: {top_probabilities[j].item():.4f})")
    print()


Input: 
너 없는 지금도 눈부신 하늘과
눈부시게 웃는 사람들
나의 헤어짐을 모르는 세상은
슬프도록 그대로인데
시간마저 데려가지 못하게
나만은 널 보내지 못했나봐
가시처럼 깊게 박힌 기억은
아파도 아픈줄 모르고
그대 기억이 지난 사랑이
내 안을 파고 드는 가시가 되어
제발 가라고 아주 가라고
애써도 나를 괴롭히는데
아픈만큼 너를 잊게 된다면
차라리 앓고 나면 그만인데
가시처럼 깊게 박힌 기억은
아파도 아픈 줄 모르고
제발 가라고 아주 가라고
애써도 나를 괴롭히는데
너무 사랑했던 나를
크게 두려웠던 나를
미치도록 너를 그리워했던
날 이제는 놓아줘
보이지 않아 내 안에 숨어
잊으려 하면 할수록 더 아파와
제발 가라고 아주 가라고
애써도 나를 괴롭히는데

Top 1 Prediction: 후회되는 (Probability: 0.0767)
Top 2 Prediction: 괴로워하는 (Probability: 0.0726)
Top 3 Prediction: 슬픔 (Probability: 0.0664)
Top 4 Prediction: 좌절한 (Probability: 0.0534)
Top 5 Prediction: 눈물이 나는 (Probability: 0.0531)
Top 6 Prediction: 비통한 (Probability: 0.0484)
Top 7 Prediction: 외로운 (Probability: 0.0464)
Top 8 Prediction: 우울한 (Probability: 0.0459)
Top 9 Prediction: 염세적인 (Probability: 0.0372)
Top 10 Prediction: 낙담한 (Probability: 0.0245)

Input: 옛날 옛날에 남양주시 와부읍
덕소리에서 자란 한 소년이 있었어요
아이는 주머니에 아무것도 없었지만
어머니의 사랑은 가득해
항상 웃음 짓던 소년이였죠
어느 날 그 소년 앞에
금을 두른 부자가 나타났답니다
부자는 그에게 물었어요
너에게 미래를 줄 테니
지금 이 순간을 나와 바꾸지 않겠니?
나 평생 꿈만을 꿨죠
알잖아

In [20]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F

# 입력 텍스트 정의
input_texts = ["""
알잖아 너를 이토록
사랑하며 기다린 나를
뭐가 그리 바쁜지
너무 보기 힘들어
너 도대체 뭐하고 다니니
그게 아냐 이유는 묻지마
그냥 믿고 기다려 주겠니
내게도 사랑을 위한
시간이 필요해
널 받아들일 수 있게
일부로 피하는 거니 노
삐삐쳐도 아무 소식 없는 너
싫으면 그냥 싫다고
솔직하게 말해봐
말리지마 내 이런 사랑을
너만 보면 미칠 것 같은 이맘을
누가 알겠어 웨딩드레스 입은
니 곁에 다른 사람이
난 두려워
나보다 더 멋진
그런 남자 니가 만날까봐
아니야 그렇지 않아
정말 너 하나뿐야
속는 셈치고 한번 믿어봐
내 눈에는 너무 이쁜 그녀를
자랑스레 친구에게 보여줬지
그 친구 네게 미안하다며
그녀 얘길 싸 그리다 했지
그녈 만난 많은 남자 중에
내 친구만도 여러명이야
말도 안돼 믿을 수 없어
혹시 쌍둥이
우연히 너를 보았지
다른 남자품안에 너를
한번도 볼 수 없었던
너무 행복한 미소
내 사랑은 무너져 버렸어
그게 아냐 변명이 아니라
그 남자는 나완 상관없어
잠시 나 어지러웠어
기댄 것 뿐이야
날 오해하지 말아 줘
나 역시 많은 여자들
만났다가 헤어져도 봤지만
한꺼번에 많은 여자를
만난 적은 없었어
니가 뭔데 날 아프게 하니
너 때문에 상처돼 버린 내 사랑
이제 다시는 너의 어떤 만남도
나같은 사랑 없을걸
난 두려워 나 역시 다시는
이런 사랑 할수 없을까봐
믿을 수 없겠지만은
니가 첫사랑인데
떠나버리면 어떡하라구
사랑까지는 바라지도 않을께
네 곁에 항상 있게만 해줘
제발 제발
""","""
마음 울적한 날엔 거리를 걸어보고
향기로운 칵테일에 취해도보고
한편의 시가 있는 전시회장도 가고
밤새도록 그리움에 편질 쓰고파
모차르트 피아노 협주곡 이십일번
그 음악을 내 귓가에 속삭여주며
아침 햇살 눈부심에 나를 깨워줄
그런 연인이 내게 있으면
나는 아직 순수함을 느끼고 싶어
어느 작은 우체국 앞 계단에 앉아
프리지아 꽃 향기를 내게 안겨줄
그런 연인을 만나 봤으면
마음 울적한 날엔 거리를 걸어보고
향기로운 칵테일에 취해도보고
한편의 시가 있는 전시회장도 가고
밤새도록 그리움에 편질 쓰고파
모차르트 피아노 협주곡 이십일번
그 음악을 내 귓가에 속삭여 주며
아침 햇살 눈부심에 나를 깨워줄
그런 연인이 내게 있으면
나는 아직 순수함을 느끼고 싶어
어느 작은 우체국 앞 계단에 앉아
프리지아 꽃향기를 내게 안겨줄
그런 연인을 만나 봤으면
마음 울적한 날엔 거리를 걸어보고
향기로운 칵테일에 취해도보고
한편의 시가 있는 전시회장도 가고
밤새도록 그리움에 편질 쓰고파
창밖에는 우울한 비가 내리고 있어
내 마음도 그 비따라 우울해지네
누가 내게 눈부신 사랑을 가져 줄까
이 세상은 나로 인해 아름다운데
마음 울적한 날엔 거리를 걸어보고
향기로운 칵테일에 취해도보고
한편의 시가 있는 전시회장도 가고
밤새도록 그리움에 편질 쓰고파
""","""
니가 없는 거리에는 내가 할 일이 없어서
마냥 걷다 걷다 보면 추억을 가끔 마주치지
떠오르는 너의 모습 내 살아나는 그리움 한 번에
참 잊기 힘든 사람이란 걸 또 한 번 느껴지는 하루
어디쯤에 머무는지 또 어떻게 살아가는지
걷다 보면 누가 말해줄 것 같아
이 거리가 익숙했던 우리 발걸음이 나란했던
그리운 날들 오늘 밤 나를 찾아온다
널 그리는 널 부르는 내 하루는
애태워도 마주친 추억이 반가워
날 부르는 목소리에 돌아보면
텅 빈 거리 어느새 수많은 니 모습만 가득해
막다른 길 다다라서 낯익은 벽 기대 보면
가로등 속 환히 비춰지는 고백하는 니가 보여
떠오르는 그때 모습 내 살아나는 설레임 한 번에
참 잊기 힘든 순간이란 걸 또 한 번 느껴지는 하루
아직 나를 생각할지 또 그녀도 나를 찾을지
걷다 보면 누가 말해줄 것 같아
이 거리가 익숙했던 우리 발걸음이 나란했던
그리운 날들 오늘 밤 나를 찾아온다
널 그리는 널 부르는 내 하루는
애태워도 마주친 추억이 반가워
날 부르는 목소리에 돌아보면
텅 빈 거리 어느새 수많은 니 모습만 가득해
부풀은 내 가슴이 밤하늘에 외쳐본다
이 거리는 널 기다린다고
널 그리는 널 부르는 내 하루는
애태워도 마주친 추억이 반가워
날 부르는 목소리에 돌아보면
텅 빈 거리 어느새 수많은 니 모습만
가득해
""","""
너와 나 둘이 정신없이 가는 곳
정처 없이 가는 곳, 정해지지 않은 곳
거기서 우리 (whoa-whoa) 서로를 재워주고 (whoa-whoa)
서로를 깨워주고 (whoa-whoa) 서로를 채워주고 (whoa-whoa)
Excuse me 잠시만 아직까진 우린 남
하지만 조만간 중독성을 자랑하는 장난감
지금 이 느낌적인 느낌이 통하는 느낌
녹아버릴 아이스크림
지금이 우리에게는 꿈이야
너와 나 둘이서 추는 춤이야
기분은 미친 듯이 예술이야
Ooh-eh-oh, ooh-eh-oh, ooh-eh-oh
하늘을 날아가는 기분이야
죽어도 상관없는 지금이야
심장은 터질 듯이 예술이야
Ooh-eh-oh, ooh-eh-oh, ooh-eh-oh
예술이야 (예술이야), 예술이야 (예술이야)
예술이야 (예술이야), 이런 날이 올 줄이야
예술이야 (예술이야), 예술이야 (예술이야)
Ooh-eh-oh, ooh-eh-oh, ooh-eh-oh
너와 나 둘이 (whoa-whoa) 밤새 잔을 부딪혀 (whoa-whoa)
밤새 뺨을 부비며 (whoa-whoa) 밤새도록 둘이서 (whoa-whoa)
눈이 점점 풀린다
니도 따라 풀린다
끌린다, 너에 대한 수수께끼가 풀린다
지금 이 춤에 너의 가빠진 숨에
수줍음에 you know what I mean
지금이 우리에게는 꿈이야
너와 나 둘이서 추는 춤이야
기분은 미친 듯이 예술이야
Ooh-eh-oh, ooh-eh-oh, ooh-eh-oh
하늘을 날아가는 기분이야
죽어도 상관없는 지금이야"""]

# # 모델과 토크나이저 로드
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')
# # 입력 텍스트를 토큰화
# inputs = tokenizer(input_texts, padding=True, truncation=True, return_tensors="pt").to(device)

# 예측 수행
with torch.no_grad():
    outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
    logits = outputs.logits
    probabilities = F.softmax(logits, dim=-1)

# 결과 출력 및 시각화
for i, text in enumerate(input_texts):
    print(f"Input: {text}")
    # print(f"Prediction: {id2label[predictions[i].item()]} (Probability: {probabilities[i][predictions[i]].item():.4f})")

    top_probabilities, top_indices = torch.topk(probabilities[i], 10)
    for j in range(10):
        print(f"Top {j+1} Prediction: {id2label[top_indices[j].item()]} (Probability: {top_probabilities[j].item():.4f})")
    print()


Input: 
알잖아 너를 이토록
사랑하며 기다린 나를
뭐가 그리 바쁜지
너무 보기 힘들어
너 도대체 뭐하고 다니니
그게 아냐 이유는 묻지마
그냥 믿고 기다려 주겠니
내게도 사랑을 위한
시간이 필요해
널 받아들일 수 있게
일부로 피하는 거니 노
삐삐쳐도 아무 소식 없는 너
싫으면 그냥 싫다고
솔직하게 말해봐
말리지마 내 이런 사랑을
너만 보면 미칠 것 같은 이맘을
누가 알겠어 웨딩드레스 입은
니 곁에 다른 사람이
난 두려워
나보다 더 멋진
그런 남자 니가 만날까봐
아니야 그렇지 않아
정말 너 하나뿐야
속는 셈치고 한번 믿어봐
내 눈에는 너무 이쁜 그녀를
자랑스레 친구에게 보여줬지
그 친구 네게 미안하다며
그녀 얘길 싸 그리다 했지
그녈 만난 많은 남자 중에
내 친구만도 여러명이야
말도 안돼 믿을 수 없어
혹시 쌍둥이
우연히 너를 보았지
다른 남자품안에 너를
한번도 볼 수 없었던
너무 행복한 미소
내 사랑은 무너져 버렸어
그게 아냐 변명이 아니라
그 남자는 나완 상관없어
잠시 나 어지러웠어
기댄 것 뿐이야
날 오해하지 말아 줘
나 역시 많은 여자들
만났다가 헤어져도 봤지만
한꺼번에 많은 여자를
만난 적은 없었어
니가 뭔데 날 아프게 하니
너 때문에 상처돼 버린 내 사랑
이제 다시는 너의 어떤 만남도
나같은 사랑 없을걸
난 두려워 나 역시 다시는
이런 사랑 할수 없을까봐
믿을 수 없겠지만은
니가 첫사랑인데
떠나버리면 어떡하라구
사랑까지는 바라지도 않을께
네 곁에 항상 있게만 해줘
제발 제발

Top 1 Prediction: 후회되는 (Probability: 0.0767)
Top 2 Prediction: 괴로워하는 (Probability: 0.0726)
Top 3 Prediction: 슬픔 (Probability: 0.0664)
Top 4 Prediction: 좌절한 (Probability: 0.0534)
Top 5 Prediction: 눈물이 나는 (Probability: 0.0531)
Top 6 Prediction: 비통한 (Probability: 0.0484)

# 문제점 발견 -> 감정간 유사도를 conv로는 반열할 수 없을거 같음

# 마지막단 MLP 하나 달아서 앞에거 다 freezing하고 마무리 학습
